In [174]:
import numpy as np
import matplotlib.pyplot as plt
import json
from matplotlib import rcParams
rcParams['font.family'] = 'serif'
rcParams['font.size'] = 14

Module for computations

In [175]:
import FEM_utilities as FEM

Parameters:

In [176]:
a = 1000. # lenght [mm]
b = 200.  # width  [mm]
t = 1.5   # thickness [mm]


Compute node coordinates, elements, constrained and loaded nodes

In [177]:
nx = 4
ny = 4

nodes, elements, fixed, loaded = FEM.Nodes(nx, ny, a, b)

In [178]:
nodes

array([[    1.,     0.,     0.,     0.],
       [    2.,   250.,     0.,     0.],
       [    3.,   500.,     0.,     0.],
       [    4.,   750.,     0.,     0.],
       [    5.,  1000.,     0.,     0.],
       [    6.,     0.,    50.,     0.],
       [    7.,   250.,    50.,     0.],
       [    8.,   500.,    50.,     0.],
       [    9.,   750.,    50.,     0.],
       [   10.,  1000.,    50.,     0.],
       [   11.,     0.,   100.,     0.],
       [   12.,   250.,   100.,     0.],
       [   13.,   500.,   100.,     0.],
       [   14.,   750.,   100.,     0.],
       [   15.,  1000.,   100.,     0.],
       [   16.,     0.,   150.,     0.],
       [   17.,   250.,   150.,     0.],
       [   18.,   500.,   150.,     0.],
       [   19.,   750.,   150.,     0.],
       [   20.,  1000.,   150.,     0.],
       [   21.,     0.,   200.,     0.],
       [   22.,   250.,   200.,     0.],
       [   23.,   500.,   200.,     0.],
       [   24.,   750.,   200.,     0.],
       [   25., 

In [179]:
elements

array([[  1.,   1.,   2.,   7.,   6.],
       [  2.,   2.,   3.,   8.,   7.],
       [  3.,   3.,   4.,   9.,   8.],
       [  4.,   4.,   5.,  10.,   9.],
       [  5.,   6.,   7.,  12.,  11.],
       [  6.,   7.,   8.,  13.,  12.],
       [  7.,   8.,   9.,  14.,  13.],
       [  8.,   9.,  10.,  15.,  14.],
       [  9.,  11.,  12.,  17.,  16.],
       [ 10.,  12.,  13.,  18.,  17.],
       [ 11.,  13.,  14.,  19.,  18.],
       [ 12.,  14.,  15.,  20.,  19.],
       [ 13.,  16.,  17.,  22.,  21.],
       [ 14.,  17.,  18.,  23.,  22.],
       [ 15.,  18.,  19.,  24.,  23.],
       [ 16.,  19.,  20.,  25.,  24.]])

In [180]:
fixed

array([ 1,  6, 11, 16, 21])

In [181]:
loaded

array([ 5, 10, 15, 20, 25])

## Write input file

Set parameters for simulation:

- *filename* : name of input file
- *eltype* : type of shell element (see conventions [here](http://media.3ds.com/support/documentation/product/V6R2013/en/English/MpeUsbHtml/pt06ch29s06alm17.htm))
- *matname*: name of material
- **E**, $\nu$: elastic properties (Young modulus and Poisson ratio, as *TYPE* is by default **isotropic**) 
- *elname*: **\*Elset** assigned name
- *bcname*, *loadextname*, *loadintname*: names for boundary condition, external and internal nodes loaded 
- *pa*, *pb*,*ax*, $\alpha$: coordinate points, axis and angle used to define local coordinate system (see conventions [here](http://media.3ds.com/support/documentation/product/V6R2013/en/English/MpeKeyHtml/ch15abk01.htm#usb-kws-morientation)) 
- *intpoints*: shell integration points
- *F*: force at tip of plate [N/mm]

In [182]:
filename = 'test.inp'

eltype = 'S4R' #'SC8R'

matname = 'material_1'

E = 72000. # modulus [MPa]
ν = 0.33   # Poisson's coefficient

elname = 'plate_1'

bcname = 'fixed_nodes'
loadextnodes = 'loaded_nodes_external'
loadintnodes = 'loaded_nodes_internal'

pa = [1.0,0.0,0.0]
pb = [0.0,1.0,0.0]
# pc = [0.0,0.0,0.0]
ax = 3
α = 0.0

intpoints = 5

F = 0.02   # load [N/mm]

In [183]:
outfile = open(filename, "wt")

outfile.write("** Lab 02 input file test\n")

# NODES section

outfile.write("**\n")
outfile.write("** NODES\n")
outfile.write("**\n")
outfile.write("*Node, nset = nglobal\n")

for i in range(nodes.shape[0]):
    nodestring = "{0:4d}".format(int(nodes[i,0]))
    for j in range(1,nodes.shape[1]):
        nodestring+=",{0:8}".format(nodes[i,j])
    nodestring+="\n"
    outfile.write(nodestring)

# ELEMENTS section
    
outfile.write("\n**\n")
outfile.write("** ELEMENTS\n")
outfile.write("**\n")
outfile.write("*Element, type = {0}\n".format(eltype))

for i in range(elements.shape[0]):
    elstring = "{0:4d}".format(int(elements[i,0]))
    for j in range(1,elements.shape[1]):
        elstring+=",{0:4d}".format(int(elements[i,j]))
    elstring+="\n"
    outfile.write(elstring)

# MATERIAL section

outfile.write("\n**\n")
outfile.write("** MATERIALS\n")
outfile.write("**\n")
outfile.write("*Material, name = {0}\n".format(matname))
outfile.write("{0},{1:6}\n".format(E,ν))

# SETS section
# NODES

outfile.write("\n**\n")
outfile.write("** SETS\n")
outfile.write("**\n")
outfile.write("*Nset, nset = {0}\n".format(bcname))

fix_str = "{0:4d}".format(int(fixed[0]))

for i in range(1,len(fixed)):
    fix_str+=",{0:4d}".format(int(fixed[i]))
fix_str+="\n"
outfile.write(fix_str)

if len(loaded > 2):
    outfile.write("*Nset, nset = {0}\n".format(loadintnodes))
    il_str = "{0:4d}".format(int(loaded[1]))
    for i in range(2,len(loaded)-1):
        il_str+=",{0:4d}".format(int(loaded[i]))
    il_str+="\n"
    outfile.write(il_str)
    
outfile.write("*Nset, nset = {0}\n".format(loadextnodes))
el_str = "{0:4d},{1:4d}\n".format(int(loaded[0]),int(loaded[-1]))
outfile.write(el_str)

# ELEMENTS
outfile.write("*Elset, elset = {0} generate\n".format(elname))
outfile.write("{0:4d},{1:4d},{2:4d}\n".format(1,elements.shape[0],1))

# ORIENTATION

outfile.write("\n**\n")
outfile.write("** LOCAL ORIENTATION\n")
outfile.write("**\n")
outfile.write("*orientation, name = local_orientation\n")
outfile.write("".join(str(pa+pb))[1:-1])
outfile.write("\n")
outfile.write("{0},{1:4}\n".format(ax,α))

# SHELL PROPERTIES
outfile.write("\n**\n")
outfile.write("** LOCAL ORIENTATION\n")
outfile.write("**\n")
outfile.write("*Shell Section, elset = {0}, material = {1}, orientation = local_orientation\n" \
              .format(elname,matname))
outfile.write("{0:4},{1:4d}\n".format(t,intpoints))

# calculation steps
outfile.write("\n**\n")
outfile.write("** STEP\n")
outfile.write("**\n")

outfile.write("*Step, name = step_1\n")
outfile.write("*Static\n")

# BOUNDARY CONDITIONS

outfile.write("\n**\n")
outfile.write("** BOUNDARY CONDITIONS\n")
outfile.write("**\n")
outfile.write("*Boundary\n")
outfile.write("{0}, ENCASTRE\n".format(bcname))

# LOADS

outfile.write("\n**\n")
outfile.write("** LOADS\n")
outfile.write("**\n")
outfile.write("*Cload\n")
outfile.write("{0}, {1:2d}, {2}\n".format(loadextnodes, 3, F*b/ny/2 ))
outfile.write("{0}, {1:2d}, {2}\n".format(loadintnodes, 3, F*b/ny ))

# FIELD OUTPUT

outfile.write("\n**\n")
outfile.write("** FIELD OUTPUT\n")
outfile.write("**\n")
outfile.write("*Output, field, variable=PRESELECT\n")

# HISTORY OUTPUT

outfile.write("\n**\n")
outfile.write("** HISTORY OUTPUT\n")
outfile.write("**\n")
outfile.write("*Output, history, variable=PRESELECT\n")

outfile.write("*End Step\n\n")

outfile.close()
